In [40]:
from code.dataset import H5Dataset
from torchvision import transforms
import torch
from code.MyPytorchModel import MyPytorchModel
import torchvision
import optuna
from pytorch_lightning.callbacks import EarlyStopping
import pytorch_lightning as pl

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
#https://stackoverflow.com/questions/57310333/can-we-disable-h5py-file-locking-for-python-file-like-object
!export HDF5_USE_FILE_LOCKING='FALSE'

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/diganes')
sys.path.append('/content/drive/My Drive/diganes/code')

In [41]:
transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize(mean=[0.49191375, 0.48235852, 0.44673872], 
                          std=[0.24706447, 0.24346213, 0.26147554])])

In [49]:
train_file = 'diganes_train_dataset.h5'
val_file = 'diganes_val_dataset.h5'
test_file = 'diganes_test_dataset.h5'

dataset = {"train": H5Dataset(train_file, transform=transform), 
           "val": H5Dataset(val_file, transform=transform),
           "test": H5Dataset(test_file, transform=transform)}

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [55]:
hparams = {
    "batch_size": 64,
    "lr": 3e-4,
    "layers_to_freeze": 13
}

model = MyPytorchModel(hparams, dataset, torchvision.models.mobilenet_v2(pretrained=True))
model.to(device)

MyPytorchModel(
  (model): PretrainedClassifier(
    (feature_extractor): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNReLU(
            (0): Conv2d(16, 96, kernel_size=(1, 1), strid

In [56]:
trainer = None

early_stop_callback = EarlyStopping(
   monitor='val_loss',
   patience=5,
   verbose=True,
   mode='min'
)

trainer = pl.Trainer(
    max_epochs=20,
    early_stop_callback=early_stop_callback,
    gpus=1 if torch.cuda.is_available() else None
)

trainer.fit(model)

GPU available: False, used: False
No environment variable for node rank defined. Set as 0.

    | Name                                | Type                 | Params
-------------------------------------------------------------------------
0   | model                               | PretrainedClassifier | 2 M   
1   | model.feature_extractor             | Sequential           | 2 M   
2   | model.feature_extractor.0           | ConvBNReLU           | 928   
3   | model.feature_extractor.0.0         | Conv2d               | 864   
4   | model.feature_extractor.0.1         | BatchNorm2d          | 64    
5   | model.feature_extractor.0.2         | ReLU6                | 0     
6   | model.feature_extractor.1           | InvertedResidual     | 896   
7   | model.feature_extractor.1.conv      | Sequential           | 896   
8   | model.feature_extractor.1.conv.0    | ConvBNReLU           | 352   
9   | model.feature_extractor.1.conv.0.0  | Conv2d               | 288   
10  | model.feature_

/Users/atad/Desktop/DIGANES/diganes/code/MyPytorchModel.py:72: UserWarning: Using a target size (torch.Size([64, 1, 29])) that is different to the input size (torch.Size([64, 29])) is deprecated. Please ensure they have the same size.
  loss = F.binary_cross_entropy(out, targets)


tensor([[[0., 0., 0.,  ..., 1., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 1., 1.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 1., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 1.,  ..., 0., 0., 0.]]])
<class 'torch.Tensor'>
torch.Size([64, 1, 29])
tensor([[0., 0., 1.,  ..., 0., 1., 1.],
        [0., 0., 1.,  ..., 1., 0., 1.],
        [0., 0., 1.,  ..., 0., 0., 1.],
        ...,
        [0., 0., 1.,  ..., 0., 1., 1.],
        [1., 0., 1.,  ..., 0., 1., 1.],
        [0., 0., 1.,  ..., 0., 1., 1.]])
<class 'torch.Tensor'>
torch.Size([64, 29])


ValueError: Classification metrics can't handle a mix of unknown and multilabel-indicator targets